In [ ]:
"""
"""

from csv import reader
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


MODEL = ['AS', 'KF', 'YS']

with open('../lats.csv', 'r') as csv_file:
    csv_reader = reader(csv_file)
    lats = list(csv_reader)
with open('../lons.csv', 'r') as csv_file:
    csv_reader = reader(csv_file)
    lons = list(csv_reader)

for i in range(79):
    for j in range(79):
        lats[i][j] = float(lats[i][j])
        lons[i][j] = float(lons[i][j])

lat_s = lats[0][0]
lat_n = lats[0][-1]
lon_w = lons[0][0]
lon_e = lons[-1][0]

DATA_F = []
DATA_P = []

LOCATION = []

for i in range(len(MODEL)):
    # モデル名
    model = MODEL[i]
    # 各地点の最大値
    data_f = np.zeros((79, 79))  # 未来
    data_p = np.zeros((79, 79))  # 過去

    # 将来
    with open('../pot_csv/' + 'HFA_' + model + '_c0_POT_DATA.csv', 'r') as csv_file:
        csv_reader = reader(csv_file)
        POT_ALL = list(csv_reader)
        for index in range(79 * 10, 79 * 79):
            POT = POT_ALL[index]
            s = [float(val) for val in POT]
            if len(s) > 1:
                data_f[index // 79][index % 79] = max(s)
            else:
                data_f[index // 79][index % 79] = 0
    # 過去
    with open('../pot_csv/' + 'HPA_' + model + '_POT_DATA.csv', 'r') as csv_file:
        csv_reader = reader(csv_file)
        POT_ALL = list(csv_reader)
        for index in range(79 * 10, 79 * 79):
            POT = POT_ALL[index]
            s = [float(val) for val in POT]
            if len(s) > 1:
                data_p[index // 79][index % 79] = max(s)
            else:
                data_p[index // 79][index % 79] = 0
    
    print(model, '(将来)が最大値を取るindex', np.argmax(data_f))
    print('その最大値', np.max(data_f))
    print(model, '(現在)が最大値を取るindex', np.argmax(data_p))
    print('その最大値', np.max(data_p))
    DATA_F.append(data_f)
    DATA_P.append(data_p)
    idx = np.where(data_f >= 24.170000305175783)
    for i in range(len(idx[0])):
        LOCATION.append(79 * idx[0][i] + idx[1][i])


# 描画(LOCATIONにある場所を表示)
fig = plt.figure(figsize=(10, 8))
levels = np.arange(0, 10, 1)
plot = np.zeros((79, 79))
for idx in LOCATION:
    plot[idx // 79][idx % 79] = 100
ax1 = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
a = plt.contourf(lons, lats, plot, levels=levels, transform=ccrs.PlateCarree(), cmap='bwr')
c_bar = plt.colorbar(a)
ax1.coastlines()